In [3]:
from fredapi import Fred
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",100)

In [4]:

# Initialize Fred with your API key
fred = Fred(api_key='db61e0d65c4d2a1053221aec21822d4e')  # Replace with your actual API key securely

# Define date range
start_date = '2002-01-01'
end_date = '2024-11-28'

# Define the indicators and their series IDs
indicators = {
    'Effective Federal Funds Rate': 'FEDFUNDS',
    '10-Year Treasury Rate': 'DGS10',
    'Consumer Price Index': 'CPIAUCSL',
    'Producer Price Index': 'PPIACO',
    'Unemployment Rate': 'UNRATE',
    'Nonfarm Payroll Employment': 'PAYEMS',
    'Real GDP': 'GDPC1',
    'Housing Starts': 'HOUST',
    'Industrial Production Index': 'INDPRO',
    'M2 Money Stock': 'M2SL',
    'Crude Oil Prices': 'DCOILWTICO',
    'Retail Sales': 'RSXFS',  # Corrected series ID for Retail Sales
    'Total Business Inventories': 'BUSINV'
}

# Fetch the data with date range
economic_data = pd.DataFrame()

for name, series_id in indicators.items():
    try:
        data = fred.get_series(
            series_id,
            observation_start=start_date,
            observation_end=end_date
        )
        if data is not None and not data.empty:
            economic_data[name] = data
            print(f"Successfully fetched data for {name}")
    except Exception as e:
        print(f"Error fetching {name}: {e}")

# Convert index to datetime if not already
economic_data.index = pd.to_datetime(economic_data.index)

Successfully fetched data for Effective Federal Funds Rate
Successfully fetched data for 10-Year Treasury Rate
Successfully fetched data for Consumer Price Index
Successfully fetched data for Producer Price Index
Successfully fetched data for Unemployment Rate
Successfully fetched data for Nonfarm Payroll Employment
Successfully fetched data for Real GDP
Successfully fetched data for Housing Starts
Successfully fetched data for Industrial Production Index
Successfully fetched data for M2 Money Stock
Successfully fetched data for Crude Oil Prices
Successfully fetched data for Retail Sales
Successfully fetched data for Total Business Inventories


In [5]:
economic_data.head()

,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
2002-01-01,1.73,NaN,177.7,128.5,5.7,130853.0,14372.785,1698.0,88.4634,5461.1,NaN,256307.0,1117388.0
2002-02-01,1.74,5.02,178.0,128.4,5.7,130732.0,NaN,1829.0,88.4578,5490.4,20.40,257670.0,1111859.0
2002-03-01,1.73,4.98,178.5,129.8,5.7,130720.0,NaN,1642.0,89.1265,5502.2,22.37,257059.0,1108193.0
2002-04-01,1.75,5.44,179.3,130.8,5.9,130616.0,14460.848,1592.0,89.5507,5502.3,26.82,261333.0,1106806.0
2002-05-01,1.75,5.08,179.5,130.8,5.8,130632.0,NaN,1764.0,89.9348,5528.5,26.58,257573.0,1110725.0


In [6]:
economic_data.shape

(274, 13)

In [7]:
economic_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2002-01-01 to 2024-10-01
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Effective Federal Funds Rate  274 non-null    float64
 1   10-Year Treasury Rate         178 non-null    float64
 2   Consumer Price Index          274 non-null    float64
 3   Producer Price Index          274 non-null    float64
 4   Unemployment Rate             274 non-null    float64
 5   Nonfarm Payroll Employment    274 non-null    float64
 6   Real GDP                      91 non-null     float64
 7   Housing Starts                274 non-null    float64
 8   Industrial Production Index   274 non-null    float64
 9   M2 Money Stock                274 non-null    float64
 10  Crude Oil Prices              178 non-null    float64
 11  Retail Sales                  274 non-null    float64
 12  Total Business Inventories    273 non-null   

In [8]:

# Ensure the DataFrame is sorted by date
economic_data.sort_index(inplace=True)

# Fill missing values
def fill_missing_values(df):
    df_filled = df.copy()

    # Ensure index is DatetimeIndex
    if not isinstance(df_filled.index, pd.DatetimeIndex):
        df_filled.index = pd.to_datetime(df_filled.index)
    df_filled.sort_index(inplace=True)

    # Create Month and Year columns once
    df_filled['Month'] = df_filled.index.month
    df_filled['Year'] = df_filled.index.year

    # Process each column individually
    for column in df.columns:
        col_data = df_filled[['Year', 'Month', column]].copy()

        # Calculate monthly means
        monthly_means = col_data.groupby(['Year', 'Month'])[column].mean().rename('Monthly_Mean').reset_index()

        # Merge monthly means back into col_data
        col_data = col_data.merge(monthly_means, on=['Year', 'Month'], how='left')

        # Fill missing values with Monthly Mean
        null_mask = col_data[column].isnull()
        col_data.loc[null_mask, column] = col_data.loc[null_mask, 'Monthly_Mean']

        # Calculate yearly means
        yearly_means = col_data.groupby('Year')[column].mean().rename('Yearly_Mean').reset_index()

        # Merge yearly means into col_data
        col_data = col_data.merge(yearly_means, on='Year', how='left')

        # Fill remaining missing values with Yearly Mean
        still_null_mask = col_data[column].isnull()
        col_data.loc[still_null_mask, column] = col_data.loc[still_null_mask, 'Yearly_Mean']

        # Update the filled values back into df_filled
        df_filled.loc[:, column] = col_data[column].values

    # Drop the auxiliary columns
    df_filled.drop(['Month', 'Year'], axis=1, inplace=True)

    return df_filled


# Apply the function to fill missing values
economic_data_filled = fill_missing_values(economic_data)
if not isinstance(economic_data_filled.index, pd.DatetimeIndex):
    economic_data_filled.index = pd.to_datetime(economic_data_filled.index)
# Ensure the DataFrame is sorted by date
economic_data_filled.sort_index(inplace=True)
print(economic_data_filled.index)
# Preview the filled data
print(economic_data_filled.head())


DatetimeIndex(['2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01',
               ...
               '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01'],
              dtype='datetime64[ns]', length=274, freq=None)
            Effective Federal Funds Rate  10-Year Treasury Rate  \
2002-01-01                          1.73                4.69625   
2002-02-01                          1.74                5.02000   
2002-03-01                          1.73                4.98000   
2002-04-01                          1.75                5.44000   
2002-05-01                          1.75                5.08000   

            Consumer Price Index  Producer Price Index  Unemployment Rate  \
2002-01-01                 177.7                 128.5               

In [9]:
print(economic_data_filled.isna().sum())

Effective Federal Funds Rate    0
10-Year Treasury Rate           0
Consumer Price Index            0
Producer Price Index            0
Unemployment Rate               0
Nonfarm Payroll Employment      0
Real GDP                        0
Housing Starts                  0
Industrial Production Index     0
M2 Money Stock                  0
Crude Oil Prices                0
Retail Sales                    0
Total Business Inventories      0
dtype: int64


In [10]:
economic_data_filled

,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
2002-01-01,1.73,4.696250,177.700,128.500,5.7,130853.0,14372.785000,1698.0,88.4634,5461.1,25.9025,256307.0,1.117388e+06
2002-02-01,1.74,5.020000,178.000,128.400,5.7,130732.0,14472.711500,1829.0,88.4578,5490.4,20.4000,257670.0,1.111859e+06
2002-03-01,1.73,4.980000,178.500,129.800,5.7,130720.0,14472.711500,1642.0,89.1265,5502.2,22.3700,257059.0,1.108193e+06
2002-04-01,1.75,5.440000,179.300,130.800,5.9,130616.0,14460.848000,1592.0,89.5507,5502.3,26.8200,261333.0,1.106806e+06
2002-05-01,1.75,5.080000,179.500,130.800,5.8,130632.0,14472.711500,1764.0,89.9348,5528.5,26.5800,257573.0,1.110725e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-01,5.33,4.175714,313.049,255.914,4.1,158548.0,23221.394667,1329.0,103.2768,21020.1,79.0500,607986.0,2.565930e+06
2024-07-01,5.33,4.480000,313.534,257.326,4.3,158692.0,23386.733000,1262.0,102.5381,21039.4,84.7000,615960.0,2.574892e+06
2024-08-01,5.33,3.990000,314.121,255.394,4.2,158770.0,23221.394667,1379.0,103.0449,21141.3,77.7400,614535.0,2.583708e+06
2024-09-01,5.13,4.175714,314.686,252.737,4.1,158993.0,23221.394667,1353.0,102.5497,21222.7,79.0500,619411.0,2.587145e+06


In [11]:
economic_data_filled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 274 entries, 2002-01-01 to 2024-10-01
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Effective Federal Funds Rate  274 non-null    float64
 1   10-Year Treasury Rate         274 non-null    float64
 2   Consumer Price Index          274 non-null    float64
 3   Producer Price Index          274 non-null    float64
 4   Unemployment Rate             274 non-null    float64
 5   Nonfarm Payroll Employment    274 non-null    float64
 6   Real GDP                      274 non-null    float64
 7   Housing Starts                274 non-null    float64
 8   Industrial Production Index   274 non-null    float64
 9   M2 Money Stock                274 non-null    float64
 10  Crude Oil Prices              274 non-null    float64
 11  Retail Sales                  274 non-null    float64
 12  Total Business Inventories    274 non-null   

# Stock Data

In [12]:
stock_data = pd.read_parquet('sp500_50stocks_data.parquet')
# Convert index to DatetimeIndex if not already
if not isinstance(stock_data.index, pd.DatetimeIndex):
    stock_data.index = pd.to_datetime(stock_data.index)

# Sort by date
stock_data.sort_index(inplace=True)
stock_data

Ticker             AMD                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2003-01-02    6.560000    7.110000    6.470000    7.010000    7.010000   
2003-01-03    7.010000    7.190000    6.860000    6.940000    6.940000   
2003-01-06    7.050000    7.230000    7.000000    7.160000    7.160000   
2003-01-07    7.250000    7.480000    7.100000    7.170000    7.170000   
2003-01-08    7.060000    7.100000    6.630000    6.690000    6.690000   
...                ...         ...         ...         ...         ...   
2024-11-20  138.960007  140.770004  135.479996  137.600006  137.600006   
2024-11-21  138.869995  140.279999  134.929993  137.490005  137.490005   
2024-11-22  137.350006  139.130005  137.039993  138.350006  138.350006   
2024-11-25  140.490005  142.350006  139.050003  141.130005  141.130005   
2024-11-26  142.550003  142.800003  136.619995  137.720001  137.720001   

Ticker                      TSLA                                      \
Price         Volume        Open        High         Low       Close   
Date                                                                   
2003-01-02  17474800         NaN         NaN         NaN         NaN   
2003-01-03   6235700         NaN         NaN         NaN         NaN   
2003-01-06   6833200         NaN         NaN         NaN         NaN   
2003-01-07   9785100         NaN         NaN         NaN         NaN   
2003-01-08  13952300         NaN         NaN         NaN         NaN   
...              ...         ...         ...         ...         ...   
2024-11-20  28843100  345.000000  346.600006  334.299988  342.029999   
2024-11-21  29311400  343.809998  347.989990  335.279999  339.640015   
2024-11-22  21784700  341.089996  361.529999  337.700012  352.559998   
2024-11-25  30923100  360.140015  361.929993  338.200012  338.589996   
2024-11-26  32092400  341.000000  346.959991  335.660004  338.230011   

Ticker                                      PM                          \
Price        Adj Close      Volume        Open        High         Low   
Date                                                                     
2003-01-02         NaN         NaN         NaN         NaN         NaN   
2003-01-03         NaN         NaN         NaN         NaN         NaN   
2003-01-06         NaN         NaN         NaN         NaN         NaN   
2003-01-07         NaN         NaN         NaN         NaN         NaN   
2003-01-08         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2024-11-20  342.029999  66340700.0  129.619995  131.240005  128.800003   
2024-11-21  339.640015  58011700.0  130.500000  132.119995  130.410004   
2024-11-22  352.559998  89140700.0  131.740005  131.809998  129.830002   
2024-11-25  338.589996  95890900.0  130.669998  131.100006  129.160004   
2024-11-26  338.230011  62295900.0  130.729996  132.940002  130.529999   

Ticker                                                  V              \
Price            Close   Adj Close     Volume        Open        High   
Date                                                                    
2003-01-02         NaN         NaN        NaN         NaN         NaN   
2003-01-03         NaN         NaN        NaN         NaN         NaN   
2003-01-06         NaN         NaN        NaN         NaN         NaN   
2003-01-07         NaN         NaN        NaN         NaN         NaN   
2003-01-08         NaN         NaN        NaN         NaN         NaN   
...                ...         ...        ...         ...         ...   
2024-11-20  130.389999  130.389999  4172700.0  311.859985  312.290009   
2024-11-21  131.210007  131.210007  3605100.0  307.299988  310.920013   
2024-11-22  129.990005  129.990005  4108200.0  307.970001  311.589996   
2024-11-25  130.710007  130.710007  43641

In [13]:
print(f"Economic data date range: {economic_data_filled.index.min()} to {economic_data_filled.index.max()}")
print(f"Stock data date range: {stock_data.index.min()} to {stock_data.index.max()}")

Economic data date range: 2002-01-01 00:00:00 to 2024-10-01 00:00:00
Stock data date range: 2003-01-02 00:00:00 to 2024-11-26 00:00:00


In [14]:
stock_data.columns

MultiIndex([( 'AMD',      'Open'),
            ( 'AMD',      'High'),
            ( 'AMD',       'Low'),
            ( 'AMD',     'Close'),
            ( 'AMD', 'Adj Close'),
            ( 'AMD',    'Volume'),
            ('TSLA',      'Open'),
            ('TSLA',      'High'),
            ('TSLA',       'Low'),
            ('TSLA',     'Close'),
            ...
            (  'VZ',       'Low'),
            (  'VZ',     'Close'),
            (  'VZ', 'Adj Close'),
            (  'VZ',    'Volume'),
            ('AAPL',      'Open'),
            ('AAPL',      'High'),
            ('AAPL',       'Low'),
            ('AAPL',     'Close'),
            ('AAPL', 'Adj Close'),
            ('AAPL',    'Volume')],
           names=['Ticker', 'Price'], length=300)

In [15]:
isinstance(stock_data.columns, pd.MultiIndex)

True

In [16]:
stock_data.index

DatetimeIndex(['2003-01-02', '2003-01-03', '2003-01-06', '2003-01-07',
               '2003-01-08', '2003-01-09', '2003-01-10', '2003-01-13',
               '2003-01-14', '2003-01-15',
               ...
               '2024-11-13', '2024-11-14', '2024-11-15', '2024-11-18',
               '2024-11-19', '2024-11-20', '2024-11-21', '2024-11-22',
               '2024-11-25', '2024-11-26'],
              dtype='datetime64[ns]', name='Date', length=5514, freq=None)

In [17]:
economic_data_filled.index

DatetimeIndex(['2002-01-01', '2002-02-01', '2002-03-01', '2002-04-01',
               '2002-05-01', '2002-06-01', '2002-07-01', '2002-08-01',
               '2002-09-01', '2002-10-01',
               ...
               '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01',
               '2024-05-01', '2024-06-01', '2024-07-01', '2024-08-01',
               '2024-09-01', '2024-10-01'],
              dtype='datetime64[ns]', length=274, freq=None)

In [18]:
# Flatten MultiIndex columns in stock_data
stock_data.columns = ['_'.join(col).strip() for col in stock_data.columns.values]

In [19]:
economic_data_filled.index.name = 'Date'
stock_data.index.name = 'Date'

# Create a daily date range based on stock data index
daily_date_range = pd.date_range(
    start=stock_data.index.min(),
    end=stock_data.index.max(),
    freq='D'  # Daily frequency
)

# Reindex economic data to daily frequency using forward fill
economic_data_daily = economic_data_filled.reindex(daily_date_range, method='ffill')

# Ensure the index name is consistent
economic_data_daily.index.name = 'Date'

# Merge the DataFrames using the date index
combined_data = stock_data.join(economic_data_daily, how='left')

# Check for missing values after merging
missing_values = combined_data.isna().sum()
print("Missing values after merging:")
print(missing_values)


Missing values after merging:
AMD_Open                       0
AMD_High                       0
AMD_Low                        0
AMD_Close                      0
AMD_Adj Close                  0
                              ..
Industrial Production Index    0
M2 Money Stock                 0
Crude Oil Prices               0
Retail Sales                   0
Total Business Inventories     0
Length: 313, dtype: int64


In [20]:
# Optionally, forward fill remaining missing values
combined_data.ffill(inplace=True)

In [21]:
# Inspect the head of the combined DataFrame
print(combined_data.head())

# Check the DataFrame info
combined_data.info()

            AMD_Open  AMD_High  AMD_Low  AMD_Close  AMD_Adj Close  AMD_Volume  \
Date                                                                            
2003-01-02      6.56      7.11     6.47       7.01           7.01    17474800   
2003-01-03      7.01      7.19     6.86       6.94           6.94     6235700   
2003-01-06      7.05      7.23     7.00       7.16           7.16     6833200   
2003-01-07      7.25      7.48     7.10       7.17           7.17     9785100   
2003-01-08      7.06      7.10     6.63       6.69           6.69    13952300   

            TSLA_Open  TSLA_High  TSLA_Low  TSLA_Close  TSLA_Adj Close  \
Date                                                                     
2003-01-02        NaN        NaN       NaN         NaN             NaN   
2003-01-03        NaN        NaN       NaN         NaN             NaN   
2003-01-06        NaN        NaN       NaN         NaN             NaN   
2003-01-07        NaN        NaN       NaN         NaN        

In [22]:
print(combined_data.isna().any().sum())

72


In [23]:
combined_data.shape, stock_data.shape

((5514, 313), (5514, 300))

In [24]:
combined_data.head()

,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2003-01-02,6.56,7.11,6.47,7.01,7.01,17474800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.846429,0.900000,0.810714,0.817857,0.817857,8397200,18.00,18.680000,17.85,18.610001,13.193541,684200,35.000000,35.549999,34.970001,35.470001,17.341360,12798800,20.877501,21.427500,20.700001,21.4275

# 2.3 Additional Feature Engineering


In [25]:
import pandas_ta as ta

tickers = ["AAPL", "NVDA", "MSFT", "GOOG", "GOOGL", "AMZN", "META", "AVGO", "LLY", "TSLA", 
                 "WMT", "JPM", "V", "XOM", "UNH", "ORCL", "MA", "HD", "PG", "COST", "JNJ", 
                 "NFLX", "ABBV", "BAC", "KO", "CRM", "CVX", "MRK", "TMUS", "AMD", "PEP", 
                 "ACN", "LIN", "TMO", "MCD", "CSCO", "ADBE", "WFC", "IBM", "GE", "ABT", 
                 "DHR", "AXP", "MS", "CAT", "NOW", "QCOM", "PM", "ISRG", "VZ"]

print(len(combined_data.columns))

for ticker in tickers:
    close_col = f'{ticker}_Close'
    
    if close_col in combined_data.columns:
        # Calculate SMA (Simple Moving Average)
        combined_data[f'{ticker}_SMA_20'] = ta.sma(combined_data[close_col], length=20)
        
        # Calculate RSI (Relative Strength Index)
        combined_data[f'{ticker}_RSI_14'] = ta.rsi(combined_data[close_col], length=14)
        
        # Calculate MACD (Moving Average Convergence Divergence)
        macd = ta.macd(combined_data[close_col], fast=12, slow=26)
        macd_columns = [f'{ticker}_MACD', f'{ticker}_MACD_Hist', f'{ticker}_MACD_Signal']
        macd.columns = macd_columns
        combined_data = pd.concat([combined_data, macd], axis=1)
        
        # Calculate Bollinger Bands
        bbands = ta.bbands(combined_data[close_col], length=20)
        bbands_columns = [f'{ticker}_BB_Lower', f'{ticker}_BB_Middle', f'{ticker}_BB_Upper', f'{ticker}_BB_Bandwidth', f'{ticker}_BB_Percentage']
        bbands.columns = bbands_columns
        combined_data = pd.concat([combined_data, bbands], axis=1)
        
        # Calculate Momentum
        combined_data[f'{ticker}_MOM_10'] = ta.mom(combined_data[close_col], length=10)
    else:
        print(f'Column {close_col} not found in combined_data.')

print(len(combined_data.columns))
combined_data.head(1)

313
863


,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories,AAPL_SMA_20,AAPL_RSI_14,AAPL_MACD,AAPL_MACD_Hist,AAPL_MACD_Signal,AAPL_BB_Lower,AAPL_BB_Middle,AAPL_BB_Upper,AAPL_BB_Bandwidth,AAPL_BB_Percentage,AAPL_MOM_10,NVDA_SMA_20,NVDA_RSI_14,NVDA_MACD,NVDA_MACD_Hist,NVDA_MACD_Signal,NVDA_BB_Lower,NVDA_BB_Middle,NVDA_BB_Upper,NVDA_BB_Bandwidth,NVDA_BB_Percentage,NVDA_MOM_10,MSFT_SMA_20,MSFT_RSI_14,MSFT_MACD,MSFT_MACD_Hist,MSFT_MACD_Signal,MSFT_BB_Lower,MSFT_BB_Middle,MSFT_BB_Upper,MSFT_BB_Bandwidth,MSFT_BB_Percentage,MSFT_MOM_10,GOOG_SMA_20,GOOG_RSI_14,GOOG_MACD,GOOG_MACD_Hist,GOOG_MACD_Signal,GOOG_BB_Lower,GOOG_BB_Middle,GOOG_BB_Upper,GOOG_BB_Bandwidth,GOOG_BB_Percentage,GOOG_MOM_10,G

In [26]:
# Number of lags
n_lags = 5

# Initialize an empty dictionary to store lagged features
lagged_features = {}

# Create lag features for stock prices
for ticker in tickers:
    close_col = f'{ticker}_Close'
    
    if close_col in combined_data.columns:
        for lag in range(1, n_lags + 1):
            lag_col_name = f'{ticker}_Close_Lag_{lag}'
            lagged_features[lag_col_name] = combined_data[close_col].shift(lag)
    else:
        print(f'Column {close_col} not found in combined_data.')

# Create lag features for economic indicators
economic_indicators = economic_data_filled.columns.tolist()

for indicator in economic_indicators:
    if indicator in combined_data.columns:
        for lag in range(1, n_lags + 1):
            lag_col_name = f'{indicator}_Lag_{lag}'
            lagged_features[lag_col_name] = combined_data[indicator].shift(lag)
    else:
        print(f'Indicator {indicator} not found in combined_data.')

# Convert the lagged features dictionary to a DataFrame
lagged_features_df = pd.DataFrame(lagged_features, index=combined_data.index)

# Concatenate the lagged features DataFrame to the original DataFrame
combined_data = pd.concat([combined_data, lagged_features_df], axis=1)

# Optionally, defragment the DataFrame to improve performance
combined_data = combined_data.copy()

# Print the number of columns
print(f"Total number of columns: {len(combined_data.columns)}")


Total number of columns: 1178


In [27]:
combined_data.shape

(5514, 1178)

In [28]:
combined_data.dropna().head()

,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories,AAPL_SMA_20,AAPL_RSI_14,AAPL_MACD,AAPL_MACD_Hist,AAPL_MACD_Signal,AAPL_BB_Lower,AAPL_BB_Middle,AAPL_BB_Upper,AAPL_BB_Bandwidth,AAPL_BB_Percentage,AAPL_MOM_10,NVDA_SMA_20,NVDA_RSI_14,NVDA_MACD,NVDA_MACD_Hist,NVDA_MACD_Signal,NVDA_BB_Lower,NVDA_BB_Middle,NVDA_BB_Upper,NVDA_BB_Bandwidth,NVDA_BB_Percentage,NVDA_MOM_10,MSFT_SMA_20,MSFT_RSI_14,MSFT_MACD,MSFT_MACD_Hist,MSFT_MACD_Signal,MSFT_BB_Lower,MSFT_BB_Middle,MSFT_BB_Upper,MSFT_BB_Bandwidth,MSFT_BB_Percentage,MSFT_MOM_10,GOOG_SMA_20,GOOG_RSI_14,GOOG_MACD,GOOG_MACD_Hist,GOOG_MACD_Signal,GOOG_BB_Lower,GOOG_BB_Middle,GOOG_BB_Upper,GOOG_BB_Bandwidth,GOOG_BB_Percentage,GOOG_MOM_10,G

In [30]:
# Extract Date-Based Features

import holidays

# Extract day of the week
combined_data['Day_of_Week'] = combined_data.index.dayofweek  # Monday=0, Sunday=6

# Extract month
combined_data['Month'] = combined_data.index.month

# Extract quarter
combined_data['Quarter'] = combined_data.index.quarter

# Identify US holidays
us_holidays = holidays.US()
combined_data['Is_Holiday'] = combined_data.index.isin(us_holidays).astype(int)

# Identify month start and end
combined_data['Is_Month_Start'] = combined_data.index.is_month_start.astype(int)
combined_data['Is_Month_End'] = combined_data.index.is_month_end.astype(int)
combined_data.head(1)

,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories,AAPL_SMA_20,AAPL_RSI_14,AAPL_MACD,AAPL_MACD_Hist,AAPL_MACD_Signal,AAPL_BB_Lower,AAPL_BB_Middle,AAPL_BB_Upper,AAPL_BB_Bandwidth,AAPL_BB_Percentage,AAPL_MOM_10,NVDA_SMA_20,NVDA_RSI_14,NVDA_MACD,NVDA_MACD_Hist,NVDA_MACD_Signal,NVDA_BB_Lower,NVDA_BB_Middle,NVDA_BB_Upper,NVDA_BB_Bandwidth,NVDA_BB_Percentage,NVDA_MOM_10,MSFT_SMA_20,MSFT_RSI_14,MSFT_MACD,MSFT_MACD_Hist,MSFT_MACD_Signal,MSFT_BB_Lower,MSFT_BB_Middle,MSFT_BB_Upper,MSFT_BB_Bandwidth,MSFT_BB_Percentage,MSFT_MOM_10,GOOG_SMA_20,GOOG_RSI_14,GOOG_MACD,GOOG_MACD_Hist,GOOG_MACD_Signal,GOOG_BB_Lower,GOOG_BB_Middle,GOOG_BB_Upper,GOOG_BB_Bandwidth,GOOG_BB_Percentage,GOOG_MOM_10,G

In [31]:
combined_data.head()

,AMD_Open,AMD_High,AMD_Low,AMD_Close,AMD_Adj Close,AMD_Volume,TSLA_Open,TSLA_High,TSLA_Low,TSLA_Close,TSLA_Adj Close,TSLA_Volume,PM_Open,PM_High,PM_Low,PM_Close,PM_Adj Close,PM_Volume,V_Open,V_High,V_Low,V_Close,V_Adj Close,V_Volume,NFLX_Open,NFLX_High,NFLX_Low,NFLX_Close,NFLX_Adj Close,NFLX_Volume,ACN_Open,ACN_High,ACN_Low,ACN_Close,ACN_Adj Close,ACN_Volume,XOM_Open,XOM_High,XOM_Low,XOM_Close,XOM_Adj Close,XOM_Volume,UNH_Open,UNH_High,UNH_Low,UNH_Close,UNH_Adj Close,UNH_Volume,JNJ_Open,JNJ_High,JNJ_Low,JNJ_Close,JNJ_Adj Close,JNJ_Volume,PG_Open,PG_High,PG_Low,PG_Close,PG_Adj Close,PG_Volume,QCOM_Open,QCOM_High,QCOM_Low,QCOM_Close,QCOM_Adj Close,QCOM_Volume,NVDA_Open,NVDA_High,NVDA_Low,NVDA_Close,NVDA_Adj Close,NVDA_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Adj Close,TMUS_Volume,JPM_Open,JPM_High,JPM_Low,JPM_Close,JPM_Adj Close,JPM_Volume,META_Open,META_High,META_Low,META_Close,META_Adj Close,META_Volume,ABT_Open,ABT_High,ABT_Low,ABT_Close,ABT_Adj Close,ABT_Volume,ISRG_Open,ISRG_High,ISRG_Low,ISRG_Close,ISRG_Adj Close,ISRG_Volume,MSFT_Open,MSFT_High,MSFT_Low,MSFT_Close,MSFT_Adj Close,MSFT_Volume,LIN_Open,LIN_High,LIN_Low,LIN_Close,LIN_Adj Close,LIN_Volume,ADBE_Open,ADBE_High,ADBE_Low,ADBE_Close,ADBE_Adj Close,ADBE_Volume,ORCL_Open,ORCL_High,ORCL_Low,ORCL_Close,ORCL_Adj Close,ORCL_Volume,PEP_Open,PEP_High,PEP_Low,PEP_Close,PEP_Adj Close,PEP_Volume,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Adj Close,DHR_Volume,ABBV_Open,ABBV_High,ABBV_Low,ABBV_Close,ABBV_Adj Close,ABBV_Volume,KO_Open,KO_High,KO_Low,KO_Close,KO_Adj Close,KO_Volume,IBM_Open,IBM_High,IBM_Low,IBM_Close,IBM_Adj Close,IBM_Volume,GOOG_Open,GOOG_High,GOOG_Low,GOOG_Close,GOOG_Adj Close,GOOG_Volume,NOW_Open,NOW_High,NOW_Low,NOW_Close,NOW_Adj Close,NOW_Volume,CRM_Open,CRM_High,CRM_Low,CRM_Close,CRM_Adj Close,CRM_Volume,CAT_Open,CAT_High,CAT_Low,CAT_Close,CAT_Adj Close,CAT_Volume,AMZN_Open,AMZN_High,AMZN_Low,AMZN_Close,AMZN_Adj Close,AMZN_Volume,GOOGL_Open,GOOGL_High,GOOGL_Low,GOOGL_Close,GOOGL_Adj Close,GOOGL_Volume,BAC_Open,BAC_High,BAC_Low,BAC_Close,BAC_Adj Close,BAC_Volume,TMO_Open,TMO_High,TMO_Low,TMO_Close,TMO_Adj Close,TMO_Volume,MS_Open,MS_High,MS_Low,MS_Close,MS_Adj Close,MS_Volume,GE_Open,GE_High,GE_Low,GE_Close,GE_Adj Close,GE_Volume,WFC_Open,WFC_High,WFC_Low,WFC_Close,WFC_Adj Close,WFC_Volume,CSCO_Open,CSCO_High,CSCO_Low,CSCO_Close,CSCO_Adj Close,CSCO_Volume,MA_Open,MA_High,MA_Low,MA_Close,MA_Adj Close,MA_Volume,MCD_Open,MCD_High,MCD_Low,MCD_Close,MCD_Adj Close,MCD_Volume,LLY_Open,LLY_High,LLY_Low,LLY_Close,LLY_Adj Close,LLY_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Adj Close,WMT_Volume,COST_Open,COST_High,COST_Low,COST_Close,COST_Adj Close,COST_Volume,AVGO_Open,AVGO_High,AVGO_Low,AVGO_Close,AVGO_Adj Close,AVGO_Volume,AXP_Open,AXP_High,AXP_Low,AXP_Close,AXP_Adj Close,AXP_Volume,CVX_Open,CVX_High,CVX_Low,CVX_Close,CVX_Adj Close,CVX_Volume,MRK_Open,MRK_High,MRK_Low,MRK_Close,MRK_Adj Close,MRK_Volume,HD_Open,HD_High,HD_Low,HD_Close,HD_Adj Close,HD_Volume,VZ_Open,VZ_High,VZ_Low,VZ_Close,VZ_Adj Close,VZ_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_Volume,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories,AAPL_SMA_20,AAPL_RSI_14,AAPL_MACD,AAPL_MACD_Hist,AAPL_MACD_Signal,AAPL_BB_Lower,AAPL_BB_Middle,AAPL_BB_Upper,AAPL_BB_Bandwidth,AAPL_BB_Percentage,AAPL_MOM_10,NVDA_SMA_20,NVDA_RSI_14,NVDA_MACD,NVDA_MACD_Hist,NVDA_MACD_Signal,NVDA_BB_Lower,NVDA_BB_Middle,NVDA_BB_Upper,NVDA_BB_Bandwidth,NVDA_BB_Percentage,NVDA_MOM_10,MSFT_SMA_20,MSFT_RSI_14,MSFT_MACD,MSFT_MACD_Hist,MSFT_MACD_Signal,MSFT_BB_Lower,MSFT_BB_Middle,MSFT_BB_Upper,MSFT_BB_Bandwidth,MSFT_BB_Percentage,MSFT_MOM_10,GOOG_SMA_20,GOOG_RSI_14,GOOG_MACD,GOOG_MACD_Hist,GOOG_MACD_Signal,GOOG_BB_Lower,GOOG_BB_Middle,GOOG_BB_Upper,GOOG_BB_Bandwidth,GOOG_BB_Percentage,GOOG_MOM_10,G

In [32]:
[col for col in combined_data.columns.tolist() if 'AMD' in col]

['AMD_Open',
 'AMD_High',
 'AMD_Low',
 'AMD_Close',
 'AMD_Adj Close',
 'AMD_Volume',
 'AMD_SMA_20',
 'AMD_RSI_14',
 'AMD_MACD',
 'AMD_MACD_Hist',
 'AMD_MACD_Signal',
 'AMD_BB_Lower',
 'AMD_BB_Middle',
 'AMD_BB_Upper',
 'AMD_BB_Bandwidth',
 'AMD_BB_Percentage',
 'AMD_MOM_10',
 'AMD_Close_Lag_1',
 'AMD_Close_Lag_2',
 'AMD_Close_Lag_3',
 'AMD_Close_Lag_4',
 'AMD_Close_Lag_5']

In [33]:
print(tickers)

['AAPL', 'NVDA', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'META', 'AVGO', 'LLY', 'TSLA', 'WMT', 'JPM', 'V', 'XOM', 'UNH', 'ORCL', 'MA', 'HD', 'PG', 'COST', 'JNJ', 'NFLX', 'ABBV', 'BAC', 'KO', 'CRM', 'CVX', 'MRK', 'TMUS', 'AMD', 'PEP', 'ACN', 'LIN', 'TMO', 'MCD', 'CSCO', 'ADBE', 'WFC', 'IBM', 'GE', 'ABT', 'DHR', 'AXP', 'MS', 'CAT', 'NOW', 'QCOM', 'PM', 'ISRG', 'VZ']


In [34]:
all_dfs = {}  # Dictionary to store DataFrames

for tick in tickers:
    # Create a dynamic DataFrame name
    df_name = "df_" + tick
    
    # Filter the combined_data DataFrame for columns matching the ticker
    globals()[df_name] = combined_data.filter(like=tick, axis=1)
    
    # Store the dynamically created DataFrame in the dictionary
    all_dfs[df_name] = globals()[df_name]


In [35]:
print(all_dfs.keys())

dict_keys(['df_AAPL', 'df_NVDA', 'df_MSFT', 'df_GOOG', 'df_GOOGL', 'df_AMZN', 'df_META', 'df_AVGO', 'df_LLY', 'df_TSLA', 'df_WMT', 'df_JPM', 'df_V', 'df_XOM', 'df_UNH', 'df_ORCL', 'df_MA', 'df_HD', 'df_PG', 'df_COST', 'df_JNJ', 'df_NFLX', 'df_ABBV', 'df_BAC', 'df_KO', 'df_CRM', 'df_CVX', 'df_MRK', 'df_TMUS', 'df_AMD', 'df_PEP', 'df_ACN', 'df_LIN', 'df_TMO', 'df_MCD', 'df_CSCO', 'df_ADBE', 'df_WFC', 'df_IBM', 'df_GE', 'df_ABT', 'df_DHR', 'df_AXP', 'df_MS', 'df_CAT', 'df_NOW', 'df_QCOM', 'df_PM', 'df_ISRG', 'df_VZ'])


In [37]:
for comp in list(all_dfs.keys()):  # Use list to index dictionary keys
    name = comp.split("_")  # Split the key name (e.g., "df_AAPL")
    comp_name = name[1]  # Extract the company name (e.g., "AAPL")
    
    # Add forwarded columns using .loc to avoid warnings
    for i in range(1, 6):  # Forwarded columns for 1 to 5 steps
        all_dfs[comp].loc[:, f'{comp_name}_target_{i}'] = all_dfs[comp][f'{comp_name}_Close'].shift(-i)


/tmp/ipykernel_255531/765470610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs[comp].loc[:, f'{comp_name}_target_{i}'] = all_dfs[comp][f'{comp_name}_Close'].shift(-i)
/tmp/ipykernel_255531/765470610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_dfs[comp].loc[:, f'{comp_name}_target_{i}'] = all_dfs[comp][f'{comp_name}_Close'].shift(-i)
/tmp/ipykernel_255531/765470610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [38]:
# Select the first five columns
first_five_columns = df_AAPL.iloc[:, :5]

# Select the last five columns
last_five_columns = df_AAPL.iloc[:, -5:]

# Combine the two subsets if you want them together
selected_columns = pd.concat([first_five_columns, last_five_columns], axis=1)

selected_columns

,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Adj Close,AAPL_target_1,AAPL_target_2,AAPL_target_3,AAPL_target_4,AAPL_target_5
Date,,,,,,,,,,
2003-01-02,0.256429,0.266429,0.256250,0.264286,0.222938,0.266071,0.266071,0.265179,0.259821,0.262143
2003-01-03,0.264286,0.266607,0.260536,0.266071,0.224443,0.266071,0.265179,0.259821,0.262143,0.262857
2003-01-06,0.268393,0.274643,0.265714,0.266071,0.224443,0.265179,0.259821,0.262143,0.262857,0.261250
2003-01-07,0.264107,0.267857,0.258393,0.265179,0.223691,0.259821,0.262143,0.262857,0.261250,0.260893
2003-01-08,0.260357,0.262679,0.257857,0.259821,0.219171,0.262143,0.262857,0.261250,0.260893,0.257679
...,...,...,...,...,...,...,...,...,...,...
2024-11-20,228.059998,229.929993,225.889999,229.000000,229.000000,228.520004,229.869995,232.869995,235.059998,NaN
2024-11-21,228.880005,230.160004,225.710007,228.520004,228.520004,229.869995,232.869995,235.059998,NaN,NaN
2024-11-22,228.059998,230.720001,228.059998,229.869995,229.869995,232.869995,235.059998,NaN,NaN,NaN


In [42]:
# Remove null values from all DataFrames
for key in all_dfs.keys():
    all_dfs[key] = all_dfs[key].dropna()

# Verify the operation
for key, df in all_dfs.items():
    print(f"{key} has {df.isna().sum().sum()} null values remaining.")


df_AAPL has 0 null values remaining.
df_NVDA has 0 null values remaining.
df_MSFT has 0 null values remaining.
df_GOOG has 0 null values remaining.
df_GOOGL has 0 null values remaining.
df_AMZN has 0 null values remaining.
df_META has 0 null values remaining.
df_AVGO has 0 null values remaining.
df_LLY has 0 null values remaining.
df_TSLA has 0 null values remaining.
df_WMT has 0 null values remaining.
df_JPM has 0 null values remaining.
df_V has 0 null values remaining.
df_XOM has 0 null values remaining.
df_UNH has 0 null values remaining.
df_ORCL has 0 null values remaining.
df_MA has 0 null values remaining.
df_HD has 0 null values remaining.
df_PG has 0 null values remaining.
df_COST has 0 null values remaining.
df_JNJ has 0 null values remaining.
df_NFLX has 0 null values remaining.
df_ABBV has 0 null values remaining.
df_BAC has 0 null values remaining.
df_KO has 0 null values remaining.
df_CRM has 0 null values remaining.
df_CVX has 0 null values remaining.
df_MRK has 0 null va

In [61]:
for key, df in all_dfs.items():
    print(all_dfs[key].isna().sum())

AAPL_Open                       0
AAPL_High                       0
AAPL_Low                        0
AAPL_Close                      0
AAPL_Adj Close                  0
AAPL_Volume                     0
AAPL_SMA_20                     0
AAPL_RSI_14                     0
AAPL_MACD                       0
AAPL_MACD_Hist                  0
AAPL_MACD_Signal                0
AAPL_BB_Lower                   0
AAPL_BB_Middle                  0
AAPL_BB_Upper                   0
AAPL_BB_Bandwidth               0
AAPL_BB_Percentage              0
AAPL_MOM_10                     0
AAPL_Close_Lag_1                0
AAPL_Close_Lag_2                0
AAPL_Close_Lag_3                0
AAPL_Close_Lag_4                0
AAPL_Close_Lag_5                0
AAPL_target_1                   0
AAPL_target_2                   0
AAPL_target_3                   0
AAPL_target_4                   0
AAPL_target_5                   0
Effective Federal Funds Rate    0
10-Year Treasury Rate           0
Consumer Price

In [47]:
economic_data_filled.head()

,Effective Federal Funds Rate,10-Year Treasury Rate,Consumer Price Index,Producer Price Index,Unemployment Rate,Nonfarm Payroll Employment,Real GDP,Housing Starts,Industrial Production Index,M2 Money Stock,Crude Oil Prices,Retail Sales,Total Business Inventories
Date,,,,,,,,,,,,,
2002-01-01,1.73,4.69625,177.7,128.5,5.7,130853.0,14372.7850,1698.0,88.4634,5461.1,25.9025,256307.0,1117388.0
2002-02-01,1.74,5.02000,178.0,128.4,5.7,130732.0,14472.7115,1829.0,88.4578,5490.4,20.4000,257670.0,1111859.0
2002-03-01,1.73,4.98000,178.5,129.8,5.7,130720.0,14472.7115,1642.0,89.1265,5502.2,22.3700,257059.0,1108193.0
2002-04-01,1.75,5.44000,179.3,130.8,5.9,130616.0,14460.8480,1592.0,89.5507,5502.3,26.8200,261333.0,1106806.0
2002-05-01,1.75,5.08000,179.5,130.8,5.8,130632.0,14472.7115,1764.0,89.9348,5528.5,26.5800,257573.0,1110725.0


In [52]:
economic_data_filled.index.name = 'Date'
for keys, dfs in all_dfs.items():        

    dfs.index.name = 'Date'
    # Create a daily date range based on stock data index
    daily_date_range = pd.date_range(
        start=dfs.index.min(),
        end=dfs.index.max(),
        freq='D'  # Daily frequency
    )

    # Reindex economic data to daily frequency using forward fill
    economic_data_daily = economic_data_filled.reindex(daily_date_range, method='ffill')

    # Ensure the index name is consistent
    economic_data_daily.index.name = 'Date'

    # Merge the DataFrames using the date index
    comb_data = dfs.join(economic_data_daily, how='left')

    # Check for missing values after merging
    missing_values = comb_data.isna().sum()
    print("Missing values after merging:")
    print(missing_values)
    all_dfs[keys] = comb_data




Missing values after merging:
AAPL_Open                       0
AAPL_High                       0
AAPL_Low                        0
AAPL_Close                      0
AAPL_Adj Close                  0
AAPL_Volume                     0
AAPL_SMA_20                     0
AAPL_RSI_14                     0
AAPL_MACD                       0
AAPL_MACD_Hist                  0
AAPL_MACD_Signal                0
AAPL_BB_Lower                   0
AAPL_BB_Middle                  0
AAPL_BB_Upper                   0
AAPL_BB_Bandwidth               0
AAPL_BB_Percentage              0
AAPL_MOM_10                     0
AAPL_Close_Lag_1                0
AAPL_Close_Lag_2                0
AAPL_Close_Lag_3                0
AAPL_Close_Lag_4                0
AAPL_Close_Lag_5                0
AAPL_target_1                   0
AAPL_target_2                   0
AAPL_target_3                   0
AAPL_target_4                   0
AAPL_target_5                   0
Effective Federal Funds Rate    0
10-Year Treasury R